<a href="https://colab.research.google.com/github/mgencler/qmem/blob/main/qbellek_temiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install qiskit qiskit-aer plotnine matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import scipy
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate

# --- Helper functions from the previous iterations ---

def jordan_wigner(pauli):
    """Perform Jordan-Wigner transformation"""
    qubit_op = SparsePauliOp(pauli, coeffs=[1])
    jw_op = qubit_op.to_matrix()
    return jw_op

def calculate_boundary_operator(simplices):
    """calculates the boundary operator of a simplicial complex"""
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    b_op = np.zeros((2**num_vertices, 2**num_vertices), dtype=complex)
    for simplex in simplices:
        if len(simplex) == 1:
            continue
        for i in range(len(simplex)):
            new_simplex = simplex[:i] + simplex[i+1:]
            sign = (-1)**i
            bit_rep_1 = 0
            bit_rep_2 = 0
            for index, value in enumerate(simplex):
                bit_rep_1 += 2**value
            for index, value in enumerate(new_simplex):
                bit_rep_2 += 2**value
            b_op[bit_rep_1, bit_rep_2] = sign
    return b_op

def create_combinatorial_laplacian(simplices, k):
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    boundary_op = calculate_boundary_operator(simplices)
    boundary_op_hermitian = boundary_op + np.transpose(np.conj(boundary_op))
    projector = np.zeros((2**num_vertices, 2**num_vertices))
    for simplex in simplices:
        if len(simplex) == k+1:
             bit_rep = 0
             for value in simplex:
                 bit_rep += 2**value
             projector[bit_rep, bit_rep] = 1
    laplacian = projector @ boundary_op_hermitian @ projector
    return laplacian

def create_HSC_gate(theta):
    """Creates the HSC gate."""
    pauli_xy = jordan_wigner('XY')
    pauli_zI = jordan_wigner('ZI')
    H = pauli_xy + pauli_zI
    HSC_matrix = scipy.linalg.expm(-1j * theta * H)
    return UnitaryGate(HSC_matrix, label=f'HSC({theta:.2f})')


def create_FEO_gate(phi):
    """Creates the FEO gate."""
    pauli_xz = jordan_wigner('XZ')
    I = np.eye(4)
    FEO_matrix = np.cos(phi) * I + 1j * np.sin(phi) * pauli_xz
    return UnitaryGate(FEO_matrix, label=f'FEO({phi:.2f})')

def create_NuBell_alpha_improved_HSC(theta=np.pi/3, phi=np.pi/6):
    """Creates the NuBell-α circuit."""
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    feo_gate = create_FEO_gate(phi)
    qc.append(feo_gate,[0,1])
    hsc_gate_improved = create_HSC_gate(theta)
    qc.append(hsc_gate_improved,[0,1])
    return qc


def simulate_quantum_circuit_with_noise(qc, shots=8192, noise_model=None):
    """Simulates the quantum circuit with optional noise model."""
    backend = Aer.get_backend('qasm_simulator')
    result = backend.run(qc, shots=shots, noise_model=noise_model).result()
    counts = result.get_counts()
    num_qubits = qc.num_qubits
    probs = np.zeros(2**num_qubits)
    for bitstring, count in counts.items():
      index = int(bitstring, 2)
      probs[index] = count / shots
    return probs


def get_density_matrix_from_probabilities(probs):
    """Creates a density matrix from measurement probabilities."""
    return np.diag(probs)

def calculate_frobenius_norm(rho_ideal, rho_real):
  """Calculates the Frobenius norm between two density matrices."""
  return np.linalg.norm(rho_ideal - rho_real, 'fro')

def calculate_shannon_entropy(probs):
    """Calculates the Shannon entropy from probabilities."""
    return -np.sum(probs * np.log2(probs + 1e-10))


def calculate_TEM(qc, shots=8192, noise_model=None):
    ideal_sv = Statevector(qc)
    rho_ideal = ideal_sv.to_operator().data
    probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
    rho_real = get_density_matrix_from_probabilities(probs)
    d_top = 0.5 * calculate_frobenius_norm(rho_ideal, rho_real)
    entropy = calculate_shannon_entropy(probs)
    TEM = 0.7 * d_top + 0.3 * entropy
    return TEM


# --- New code for spectral approach ---

def get_cartan_decomposition_simplified(laplacian):
  """Simplified Cartan decomposition for a 2x2 matrix."""
  eigvals, eigvecs = np.linalg.eigh(laplacian)
  return eigvals, eigvecs

def simulate_time_evolution(laplacian, time, initial_state):
    """Simulates the time evolution."""
    eigvals, eigvecs = get_cartan_decomposition_simplified(laplacian)
    evolved_state = np.zeros_like(initial_state, dtype=complex)
    for i, eigenvalue in enumerate(eigvals):
      evolved_state += np.exp(-1j * eigenvalue * time) *  (np.conjugate(eigvecs[:, i]) @ initial_state) *  eigvecs[:, i]
    return evolved_state

def measure_trace_evolution(laplacian, num_times, shots=8192, noise_model=None):
  """Measures the trace evolution of a laplacian."""
  num_vertices = laplacian.shape[0]
  initial_state = np.zeros(num_vertices)
  initial_state[0] = 1
  trace_values = np.zeros(num_times, dtype=complex)
  for t_index, time in enumerate(np.linspace(0, 2*np.pi, num_times)):
    evolved_state = simulate_time_evolution(laplacian, time, initial_state)
    trace_value = np.trace(np.outer(evolved_state, np.conj(evolved_state)))
    trace_values[t_index] = trace_value
  return trace_values.real #trace is real for the laplacian

def interpolate_signal(signal, num_points, type="trigonometric"):
    """Interpolates the given signal using DFT based techniques."""
    N = len(signal)
    if type == "trigonometric":
        M = N
        padded_signal = np.pad(signal, ((0, max(0, M-N))))
        X = np.fft.fft(padded_signal)
        freq = np.fft.fftfreq(M)
        time_points = np.linspace(0, 2 * np.pi, num_points)
        interpolated_signal = np.zeros_like(time_points, dtype=complex)
        for n, time in enumerate(time_points):
          for k, f_freq in enumerate(freq):
            interpolated_signal[n] += X[k] * np.exp(2*np.pi * 1j * f_freq * time) / M
        return interpolated_signal
    else:
        raise ValueError("Invalid type")


def compute_spectrum(trace_values, num_coefficients):
    """Computes the spectrum from the trace evolution by interpolation and DFT."""
    interpolated_signal = interpolate_signal(trace_values, num_points=len(trace_values))
    coefficients = np.fft.fft(interpolated_signal)
    return coefficients[:num_coefficients]

def calculate_spectral_distance(ideal_spectrum, real_spectrum):
  """Calculates the spectral distance based on spectral coefficients."""
  return np.linalg.norm(ideal_spectrum - real_spectrum)

def calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
    """Calculates the topological error based on the spectral norm, given a circuit"""
    ideal_trace = measure_trace_evolution(laplacian_ideal, num_times=num_times)
    real_trace = measure_trace_evolution(laplacian_real, num_times=num_times)

    ideal_spectrum = compute_spectrum(ideal_trace, num_coefficients=num_coefficients)
    real_spectrum = compute_spectrum(real_trace, num_coefficients=num_coefficients)
    return calculate_spectral_distance(ideal_spectrum, real_spectrum)


def calculate_TEM_spectral(qc, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
  """Calculates the TEM with the spectral metric instead of the Frobenius norm."""
  ideal_sv = Statevector(qc)
  ideal_state_matrix = ideal_sv.to_operator().data
  probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
  rho_real = get_density_matrix_from_probabilities(probs)

  simplices = [(0,), (1,), (0, 1)]
  laplacian_ideal = create_combinatorial_laplacian(simplices, 1)
  laplacian_real  = create_combinatorial_laplacian(simplices, 1)
  d_topo = calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=num_times, num_coefficients=num_coefficients)
  entropy = calculate_shannon_entropy(probs)
  TEM = 0.7 * d_topo + 0.3 * entropy
  return TEM


# Example Usage
if __name__ == '__main__':
  qc = create_NuBell_alpha_improved_HSC()
  TEM_frobenius = calculate_TEM(qc)
  TEM_spectral = calculate_TEM_spectral(qc)
  print(f"TEM with Frobenius norm: {TEM_frobenius}")
  print(f"TEM with spectral metric: {TEM_spectral}")

QiskitError: 'No counts for experiment "0"'

In [5]:
import numpy as np
import scipy
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate

# --- Helper functions from the previous iterations ---

def jordan_wigner(pauli):
    """Perform Jordan-Wigner transformation"""
    qubit_op = SparsePauliOp(pauli, coeffs=[1])
    jw_op = qubit_op.to_matrix()
    return jw_op

def calculate_boundary_operator(simplices):
    """calculates the boundary operator of a simplicial complex"""
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    b_op = np.zeros((2**num_vertices, 2**num_vertices), dtype=complex)
    for simplex in simplices:
        if len(simplex) == 1:
            continue
        for i in range(len(simplex)):
            new_simplex = simplex[:i] + simplex[i+1:]
            sign = (-1)**i
            bit_rep_1 = 0
            bit_rep_2 = 0
            for index, value in enumerate(simplex):
                bit_rep_1 += 2**value
            for index, value in enumerate(new_simplex):
                bit_rep_2 += 2**value
            b_op[bit_rep_1, bit_rep_2] = sign
    return b_op

def create_combinatorial_laplacian(simplices, k):
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    boundary_op = calculate_boundary_operator(simplices)
    boundary_op_hermitian = boundary_op + np.transpose(np.conj(boundary_op))
    projector = np.zeros((2**num_vertices, 2**num_vertices))
    for simplex in simplices:
        if len(simplex) == k+1:
             bit_rep = 0
             for value in simplex:
                 bit_rep += 2**value
             projector[bit_rep, bit_rep] = 1
    laplacian = projector @ boundary_op_hermitian @ projector
    return laplacian

def create_HSC_gate(theta):
    """Creates the HSC gate."""
    pauli_xy = jordan_wigner('XY')
    pauli_zI = jordan_wigner('ZI')
    H = pauli_xy + pauli_zI
    HSC_matrix = scipy.linalg.expm(-1j * theta * H)
    return UnitaryGate(HSC_matrix, label=f'HSC({theta:.2f})')


def create_FEO_gate(phi):
    """Creates the FEO gate."""
    pauli_xz = jordan_wigner('XZ')
    I = np.eye(4)
    FEO_matrix = np.cos(phi) * I + 1j * np.sin(phi) * pauli_xz
    return UnitaryGate(FEO_matrix, label=f'FEO({phi:.2f})')

def create_NuBell_alpha_improved_HSC(theta=np.pi/3, phi=np.pi/6):
    """Creates the NuBell-α circuit."""
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    feo_gate = create_FEO_gate(phi)
    qc.append(feo_gate,[0,1])
    hsc_gate_improved = create_HSC_gate(theta)
    qc.append(hsc_gate_improved,[0,1])
    qc.measure_all() # add measurements
    return qc


def simulate_quantum_circuit_with_noise(qc, shots=8192, noise_model=None):
    """Simulates the quantum circuit with optional noise model."""
    backend = Aer.get_backend('aer_simulator') #use aer simulator
    result = backend.run(qc, shots=shots, noise_model=noise_model).result()
    counts = result.get_counts()
    num_qubits = qc.num_qubits
    probs = np.zeros(2**num_qubits)
    for bitstring, count in counts.items():
      index = int(bitstring, 2)
      probs[index] = count / shots
    return probs


def get_density_matrix_from_probabilities(probs):
    """Creates a density matrix from measurement probabilities."""
    return np.diag(probs)

def calculate_frobenius_norm(rho_ideal, rho_real):
  """Calculates the Frobenius norm between two density matrices."""
  return np.linalg.norm(rho_ideal - rho_real, 'fro')

def calculate_shannon_entropy(probs):
    """Calculates the Shannon entropy from probabilities."""
    return -np.sum(probs * np.log2(probs + 1e-10))


def calculate_TEM(qc, shots=8192, noise_model=None):
    ideal_sv = Statevector(qc)
    rho_ideal = ideal_sv.to_operator().data
    probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
    rho_real = get_density_matrix_from_probabilities(probs)
    d_top = 0.5 * calculate_frobenius_norm(rho_ideal, rho_real)
    entropy = calculate_shannon_entropy(probs)
    TEM = 0.7 * d_top + 0.3 * entropy
    return TEM


# --- New code for spectral approach ---

def get_cartan_decomposition_simplified(laplacian):
  """Simplified Cartan decomposition for a 2x2 matrix."""
  eigvals, eigvecs = np.linalg.eigh(laplacian)
  return eigvals, eigvecs

def simulate_time_evolution(laplacian, time, initial_state):
    """Simulates the time evolution."""
    eigvals, eigvecs = get_cartan_decomposition_simplified(laplacian)
    evolved_state = np.zeros_like(initial_state, dtype=complex)
    for i, eigenvalue in enumerate(eigvals):
      evolved_state += np.exp(-1j * eigenvalue * time) *  (np.conjugate(eigvecs[:, i]) @ initial_state) *  eigvecs[:, i]
    return evolved_state

def measure_trace_evolution(laplacian, num_times, shots=8192, noise_model=None):
  """Measures the trace evolution of a laplacian."""
  num_vertices = laplacian.shape[0]
  initial_state = np.zeros(num_vertices)
  initial_state[0] = 1
  trace_values = np.zeros(num_times, dtype=complex)
  for t_index, time in enumerate(np.linspace(0, 2*np.pi, num_times)):
    evolved_state = simulate_time_evolution(laplacian, time, initial_state)
    trace_value = np.trace(np.outer(evolved_state, np.conj(evolved_state)))
    trace_values[t_index] = trace_value
  return trace_values.real #trace is real for the laplacian

def interpolate_signal(signal, num_points, type="trigonometric"):
    """Interpolates the given signal using DFT based techniques."""
    N = len(signal)
    if type == "trigonometric":
        M = N
        padded_signal = np.pad(signal, ((0, max(0, M-N))))
        X = np.fft.fft(padded_signal)
        freq = np.fft.fftfreq(M)
        time_points = np.linspace(0, 2 * np.pi, num_points)
        interpolated_signal = np.zeros_like(time_points, dtype=complex)
        for n, time in enumerate(time_points):
          for k, f_freq in enumerate(freq):
            interpolated_signal[n] += X[k] * np.exp(2*np.pi * 1j * f_freq * time) / M
        return interpolated_signal
    else:
        raise ValueError("Invalid type")


def compute_spectrum(trace_values, num_coefficients):
    """Computes the spectrum from the trace evolution by interpolation and DFT."""
    interpolated_signal = interpolate_signal(trace_values, num_points=len(trace_values))
    coefficients = np.fft.fft(interpolated_signal)
    return coefficients[:num_coefficients]

def calculate_spectral_distance(ideal_spectrum, real_spectrum):
  """Calculates the spectral distance based on spectral coefficients."""
  return np.linalg.norm(ideal_spectrum - real_spectrum)

def calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
    """Calculates the topological error based on the spectral norm, given a circuit"""
    ideal_trace = measure_trace_evolution(laplacian_ideal, num_times=num_times)
    real_trace = measure_trace_evolution(laplacian_real, num_times=num_times)

    ideal_spectrum = compute_spectrum(ideal_trace, num_coefficients=num_coefficients)
    real_spectrum = compute_spectrum(real_trace, num_coefficients=num_coefficients)
    return calculate_spectral_distance(ideal_spectrum, real_spectrum)


def calculate_TEM_spectral(qc, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
  """Calculates the TEM with the spectral metric instead of the Frobenius norm."""
  ideal_sv = Statevector(qc)
  ideal_state_matrix = ideal_sv.to_operator().data
  probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
  rho_real = get_density_matrix_from_probabilities(probs)

  simplices = [(0,), (1,), (0, 1)]
  laplacian_ideal = create_combinatorial_laplacian(simplices, 1)
  laplacian_real  = create_combinatorial_laplacian(simplices, 1)
  d_topo = calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=num_times, num_coefficients=num_coefficients)
  entropy = calculate_shannon_entropy(probs)
  TEM = 0.7 * d_topo + 0.3 * entropy
  return TEM


# Example Usage
if __name__ == '__main__':
  qc = create_NuBell_alpha_improved_HSC()
  print("Quantum Circuit", qc)
  TEM_frobenius = calculate_TEM(qc)
  TEM_spectral = calculate_TEM_spectral(qc)
  print(f"TEM with Frobenius norm: {TEM_frobenius}")
  print(f"TEM with spectral metric: {TEM_spectral}")

Quantum Circuit         ┌───┐     ┌────────────┐┌────────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤0           ├┤0           ├─░─┤M├───
        └───┘┌─┴─┐│  FEO(0.52) ││  HSC(1.05) │ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤1           ├┤1           ├─░──╫─┤M├
             └───┘└────────────┘└────────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════╩══╩═
                                                  0  1 


QiskitError: 'Cannot apply instruction with classical bits: measure'

In [6]:
import numpy as np
import scipy
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Statevector, Operator
from qiskit_aer import Aer
from qiskit.circuit.library import UnitaryGate

# --- Helper functions from the previous iterations ---

def jordan_wigner(pauli):
    """Perform Jordan-Wigner transformation"""
    qubit_op = SparsePauliOp(pauli, coeffs=[1])
    jw_op = qubit_op.to_matrix()
    return jw_op

def calculate_boundary_operator(simplices):
    """calculates the boundary operator of a simplicial complex"""
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    b_op = np.zeros((2**num_vertices, 2**num_vertices), dtype=complex)
    for simplex in simplices:
        if len(simplex) == 1:
            continue
        for i in range(len(simplex)):
            new_simplex = simplex[:i] + simplex[i+1:]
            sign = (-1)**i
            bit_rep_1 = 0
            bit_rep_2 = 0
            for index, value in enumerate(simplex):
                bit_rep_1 += 2**value
            for index, value in enumerate(new_simplex):
                bit_rep_2 += 2**value
            b_op[bit_rep_1, bit_rep_2] = sign
    return b_op

def create_combinatorial_laplacian(simplices, k):
    num_vertices = max(max(simplex) for simplex in simplices) + 1
    boundary_op = calculate_boundary_operator(simplices)
    boundary_op_hermitian = boundary_op + np.transpose(np.conj(boundary_op))
    projector = np.zeros((2**num_vertices, 2**num_vertices))
    for simplex in simplices:
        if len(simplex) == k+1:
             bit_rep = 0
             for value in simplex:
                 bit_rep += 2**value
             projector[bit_rep, bit_rep] = 1
    laplacian = projector @ boundary_op_hermitian @ projector
    return laplacian

def create_HSC_gate(theta):
    """Creates the HSC gate."""
    pauli_xy = jordan_wigner('XY')
    pauli_zI = jordan_wigner('ZI')
    H = pauli_xy + pauli_zI
    HSC_matrix = scipy.linalg.expm(-1j * theta * H)
    return UnitaryGate(HSC_matrix, label=f'HSC({theta:.2f})')


def create_FEO_gate(phi):
    """Creates the FEO gate."""
    pauli_xz = jordan_wigner('XZ')
    I = np.eye(4)
    FEO_matrix = np.cos(phi) * I + 1j * np.sin(phi) * pauli_xz
    return UnitaryGate(FEO_matrix, label=f'FEO({phi:.2f})')


def create_NuBell_alpha_improved_HSC(theta=np.pi/3, phi=np.pi/6, measure=True):
    """Creates the NuBell-α circuit."""
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    feo_gate = create_FEO_gate(phi)
    qc.append(feo_gate,[0,1])
    hsc_gate_improved = create_HSC_gate(theta)
    qc.append(hsc_gate_improved,[0,1])
    if measure:
      qc.measure_all() # add measurements only if the circuit is used for measurement
    return qc


def simulate_quantum_circuit_with_noise(qc, shots=8192, noise_model=None):
    """Simulates the quantum circuit with optional noise model."""
    backend = Aer.get_backend('aer_simulator')
    result = backend.run(qc, shots=shots, noise_model=noise_model).result()
    counts = result.get_counts()
    num_qubits = qc.num_qubits
    probs = np.zeros(2**num_qubits)
    for bitstring, count in counts.items():
      index = int(bitstring, 2)
      probs[index] = count / shots
    return probs


def get_density_matrix_from_probabilities(probs):
    """Creates a density matrix from measurement probabilities."""
    return np.diag(probs)

def calculate_frobenius_norm(rho_ideal, rho_real):
  """Calculates the Frobenius norm between two density matrices."""
  return np.linalg.norm(rho_ideal - rho_real, 'fro')

def calculate_shannon_entropy(probs):
    """Calculates the Shannon entropy from probabilities."""
    return -np.sum(probs * np.log2(probs + 1e-10))


def calculate_TEM(qc, shots=8192, noise_model=None):
    ideal_qc = create_NuBell_alpha_improved_HSC(measure=False) # Create an ideal circuit without measurements
    ideal_sv = Statevector(ideal_qc)
    rho_ideal = ideal_sv.to_operator().data
    probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
    rho_real = get_density_matrix_from_probabilities(probs)
    d_top = 0.5 * calculate_frobenius_norm(rho_ideal, rho_real)
    entropy = calculate_shannon_entropy(probs)
    TEM = 0.7 * d_top + 0.3 * entropy
    return TEM


# --- New code for spectral approach ---

def get_cartan_decomposition_simplified(laplacian):
  """Simplified Cartan decomposition for a 2x2 matrix."""
  eigvals, eigvecs = np.linalg.eigh(laplacian)
  return eigvals, eigvecs

def simulate_time_evolution(laplacian, time, initial_state):
    """Simulates the time evolution."""
    eigvals, eigvecs = get_cartan_decomposition_simplified(laplacian)
    evolved_state = np.zeros_like(initial_state, dtype=complex)
    for i, eigenvalue in enumerate(eigvals):
      evolved_state += np.exp(-1j * eigenvalue * time) *  (np.conjugate(eigvecs[:, i]) @ initial_state) *  eigvecs[:, i]
    return evolved_state

def measure_trace_evolution(laplacian, num_times, shots=8192, noise_model=None):
  """Measures the trace evolution of a laplacian."""
  num_vertices = laplacian.shape[0]
  initial_state = np.zeros(num_vertices)
  initial_state[0] = 1
  trace_values = np.zeros(num_times, dtype=complex)
  for t_index, time in enumerate(np.linspace(0, 2*np.pi, num_times)):
    evolved_state = simulate_time_evolution(laplacian, time, initial_state)
    trace_value = np.trace(np.outer(evolved_state, np.conj(evolved_state)))
    trace_values[t_index] = trace_value
  return trace_values.real #trace is real for the laplacian

def interpolate_signal(signal, num_points, type="trigonometric"):
    """Interpolates the given signal using DFT based techniques."""
    N = len(signal)
    if type == "trigonometric":
        M = N
        padded_signal = np.pad(signal, ((0, max(0, M-N))))
        X = np.fft.fft(padded_signal)
        freq = np.fft.fftfreq(M)
        time_points = np.linspace(0, 2 * np.pi, num_points)
        interpolated_signal = np.zeros_like(time_points, dtype=complex)
        for n, time in enumerate(time_points):
          for k, f_freq in enumerate(freq):
            interpolated_signal[n] += X[k] * np.exp(2*np.pi * 1j * f_freq * time) / M
        return interpolated_signal
    else:
        raise ValueError("Invalid type")


def compute_spectrum(trace_values, num_coefficients):
    """Computes the spectrum from the trace evolution by interpolation and DFT."""
    interpolated_signal = interpolate_signal(trace_values, num_points=len(trace_values))
    coefficients = np.fft.fft(interpolated_signal)
    return coefficients[:num_coefficients]

def calculate_spectral_distance(ideal_spectrum, real_spectrum):
  """Calculates the spectral distance based on spectral coefficients."""
  return np.linalg.norm(ideal_spectrum - real_spectrum)

def calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
    """Calculates the topological error based on the spectral norm, given a circuit"""
    ideal_trace = measure_trace_evolution(laplacian_ideal, num_times=num_times)
    real_trace = measure_trace_evolution(laplacian_real, num_times=num_times)

    ideal_spectrum = compute_spectrum(ideal_trace, num_coefficients=num_coefficients)
    real_spectrum = compute_spectrum(real_trace, num_coefficients=num_coefficients)
    return calculate_spectral_distance(ideal_spectrum, real_spectrum)


def calculate_TEM_spectral(qc, num_times=50, num_coefficients=10, shots=8192, noise_model=None):
  """Calculates the TEM with the spectral metric instead of the Frobenius norm."""
  ideal_qc = create_NuBell_alpha_improved_HSC(measure=False)  # Create an ideal circuit without measurements
  ideal_sv = Statevector(ideal_qc)
  ideal_state_matrix = ideal_sv.to_operator().data
  probs = simulate_quantum_circuit_with_noise(qc, shots=shots, noise_model=noise_model)
  rho_real = get_density_matrix_from_probabilities(probs)

  simplices = [(0,), (1,), (0, 1)]
  laplacian_ideal = create_combinatorial_laplacian(simplices, 1)
  laplacian_real  = create_combinatorial_laplacian(simplices, 1)
  d_topo = calculate_topo_error_spectral(laplacian_ideal, laplacian_real, num_times=num_times, num_coefficients=num_coefficients)
  entropy = calculate_shannon_entropy(probs)
  TEM = 0.7 * d_topo + 0.3 * entropy
  return TEM


# Example Usage
if __name__ == '__main__':
  qc = create_NuBell_alpha_improved_HSC()
  print("Quantum Circuit", qc)
  TEM_frobenius = calculate_TEM(qc)
  TEM_spectral = calculate_TEM_spectral(qc)
  print(f"TEM with Frobenius norm: {TEM_frobenius}")
  print(f"TEM with spectral metric: {TEM_spectral}")

Quantum Circuit         ┌───┐     ┌────────────┐┌────────────┐ ░ ┌─┐   
   q_0: ┤ H ├──■──┤0           ├┤0           ├─░─┤M├───
        └───┘┌─┴─┐│  FEO(0.52) ││  HSC(1.05) │ ░ └╥┘┌─┐
   q_1: ─────┤ X ├┤1           ├┤1           ├─░──╫─┤M├
             └───┘└────────────┘└────────────┘ ░  ║ └╥┘
meas: 2/══════════════════════════════════════════╩══╩═
                                                  0  1 
TEM with Frobenius norm: 0.826793680139416
TEM with spectral metric: 0.5376479392795055
